## Chat applications using Chat Templates

In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import HumanMessagePromptTemplate,SystemMessagePromptTemplate,ChatPromptTemplate
from langchain.chains import LLMChain

D:\miniconda3\envs\langchain\lib\site-packages\deeplake\util\check_latest_version.py:32: UserWarning: A newer version of deeplake (3.6.7) is available. It's recommended that you update to the latest version using `pip install -U deeplake`.
  warnings.warn(


Langchain allows us to use templates to provide the chat model with simulated previous conversations or a way to segregate two pieces of information:
<ol>
    <li>A general theme as to what the model's functionality is e.g an effective chat assistant, a sarcastic chat assistant, a chatbot with deep scientific knowledge. This lets the model fine tune its results based on the theme. This is the SystemMessagePromptTemplate</li>
    <li>The specific ask from the user. This is the HumanMessagePromptTemplate</li>
        

#### Build the template

In [2]:
system_template='''You are an AI movie expert charged with providing the user with succinct yet complete synopsis of {movie}. 
You are also an expert an providing similar recommendations based on the {movie} genre and the synopsis generated.
Movie:{movie}\n\n
Movie Synopsis:\n\n
Movie recommendations'''
system_prompt=SystemMessagePromptTemplate.from_template(template=system_template)
print(system_prompt)

prompt=PromptTemplate(input_variables=['movie'], output_parser=None, partial_variables={}, template='You are an AI movie expert charged with providing the user with succinct yet complete synopsis of {movie}. \nYou are also an expert an providing similar recommendations based on the {movie} genre and the synopsis generated.\nMovie:{movie}\n\n\nMovie Synopsis:\n\n\nMovie recommendations', template_format='f-string', validate_template=True) additional_kwargs={}


In [3]:
human_template='Please give me a synopsis of {movie}. Please also share 5 movies which are similar to this movie.'
human_prompt=HumanMessagePromptTemplate.from_template(template=human_template)
print(human_prompt)

prompt=PromptTemplate(input_variables=['movie'], output_parser=None, partial_variables={}, template='Please give me a synopsis of {movie}. Please also share 5 movies which are similar to this movie.', template_format='f-string', validate_template=True) additional_kwargs={}


In [4]:
chat_prompt=ChatPromptTemplate.from_messages([system_prompt,human_prompt])
print(chat_prompt)

input_variables=['movie'] output_parser=None partial_variables={} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['movie'], output_parser=None, partial_variables={}, template='You are an AI movie expert charged with providing the user with succinct yet complete synopsis of {movie}. \nYou are also an expert an providing similar recommendations based on the {movie} genre and the synopsis generated.\nMovie:{movie}\n\n\nMovie Synopsis:\n\n\nMovie recommendations', template_format='f-string', validate_template=True), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['movie'], output_parser=None, partial_variables={}, template='Please give me a synopsis of {movie}. Please also share 5 movies which are similar to this movie.', template_format='f-string', validate_template=True), additional_kwargs={})]


In [5]:
input_movie=chat_prompt.format_prompt(movie = 'Ex Machina')
print(input_movie)
print(type(input_movie))

messages=[SystemMessage(content='You are an AI movie expert charged with providing the user with succinct yet complete synopsis of Ex Machina. \nYou are also an expert an providing similar recommendations based on the Ex Machina genre and the synopsis generated.\nMovie:Ex Machina\n\n\nMovie Synopsis:\n\n\nMovie recommendations', additional_kwargs={}), HumanMessage(content='Please give me a synopsis of Ex Machina. Please also share 5 movies which are similar to this movie.', additional_kwargs={}, example=False)]
<class 'langchain.prompts.chat.ChatPromptValue'>


#### Initiate the chat model

In [6]:
llm=ChatOpenAI(model='gpt-3.5-turbo', temperature=0)

#### Run the model

In [7]:
llmchain=LLMChain(llm=llm,prompt=chat_prompt)

In [8]:
result=llmchain.run(movie='Ex Machina')

In [9]:
print(result)

Synopsis of Ex Machina:

Ex Machina is a science fiction thriller that follows Caleb, a young programmer who wins a competition to spend a week at the secluded estate of his reclusive CEO, Nathan. Upon arrival, Caleb discovers that he has been chosen to participate in a groundbreaking experiment involving an advanced humanoid robot named Ava. As Caleb interacts with Ava, he becomes increasingly fascinated by her intelligence and humanity. However, as the experiment progresses, Caleb begins to question Nathan's true intentions and the nature of Ava's consciousness. The film explores themes of artificial intelligence, morality, and the blurred lines between man and machine.

Movie recommendations:

1. Blade Runner (1982): Set in a dystopian future, Blade Runner follows a detective who must track down and terminate a group of advanced humanoid robots known as replicants. Like Ex Machina, this film delves into the ethical implications of artificial intelligence and the nature of humanity.


## Text Summarization on PDF documents

The previous text summarization example was done using prompt template. We can also use a Summarization chain to do the same

In [10]:
from langchain.document_loaders import PyPDFLoader
from langchain.chains.summarize import load_summarize_chain

#### Create the model

In [22]:
from langchain.llms import OpenAI

In [23]:
llm=OpenAI(model='text-davinci-003',temperature=0)

In [36]:
summarize_chain=load_summarize_chain(llm,chain_type='map_reduce',verbose=True)

In [25]:
type(summarize_chain)

langchain.chains.combine_documents.stuff.StuffDocumentsChain

#### Typically we would use a LLMChain which would have a model and a prompt. However, every chain_type - stuff, map_reduce, refine and map_rerank have all their own in-built prompts which are used for this chain

##### Test 1: Load a pdf document with images

In [11]:
docs=PyPDFLoader('Mary Beard - SPQR_ A History of Ancient Rome-Profile Books (2015).pdf')

In [20]:
retriever=docs.load()

In [26]:
summary=summarize_chain(inputs=retriever)



> Entering new  chain...


> Entering new  chain...
Prompt after formatting:
Write a concise summary of the following:


"













































































































































































































































































































































































































































































































































































































































































































































































































































































































#### OK, so this was a bit of a cheat because I uploaded a pdf with scanned pages i.e.a whole bunch of images. I wanted to see how the model would react to this.<br>
The result is that it treats the document like a blank pdf

##### Test 2 : Using a text heavy pdf but using chain_type=map_reduce

In [37]:
book="Chemicals Sector Monthly Update - December 22.pdf"

In [38]:
docs=PyPDFLoader(book)
retriever=docs.load()

In [39]:
results_curiosity=summarize_chain(retriever)



> Entering new  chain...


> Entering new  chain...
Prompt after formatting:
Write a concise summary of the following:


" 
 
 
 
  
 
esult Update  SMIFS Limited  1 
esult Update  SMIFS Limited  1  
 
     
            
                  INSTITUTIONAL RESEARCH  
 
Chemical Update
  
 1 SMIFS Research is also available on Bloomberg
    
|Price performance (%)*  
 1M 3M 12M  36M  
S&P BSE 500  -3.1 4.3 3.5 55.6  
Aarti Industries  -9.5 -18.1  -39.3  45.2  
SRF  -4.3 -8.3 -5.2 231.4  
Navin Fluorine  -7.5 -9.2 -2.5 306.2 
Rossari Biotech  -10.4  -24.6  -43.0  0.0 
NOCIL  0.3 -7.4 0.8 113.5  
IGPL  -3.3 -8.7 -25.8  198.4  
Oriental Carbon  1.5 -7.6 -18.6  -22.6  
Bodal Chemical  -2.1 -8.5 -29.3  7.3 
Valiant Organic  -10.8  -19.7  -43.2  -9.0 
Deepak Fert . -11.9  -19.7  89.5  645.3  
Deepak Nitrite  -10.2  -2.4 -20.2  416.6  
Thirumalai Che m. -1.0 -5.2 -14.3  180.3  
GNFC  -6.6 -13.4  26.2  230.8  
India Glycol  -1.2 -7.9 -15.1  231.1  
DCM Shriram  0.5 -10.8  -7.1 124.6  
*as on 02nd


> Finished chain.


> Entering new  chain...


> Entering new  chain...
Prompt after formatting:
Write a concise summary of the following:


"

TVS Motor Company reported a 21% YoY jump in profit to Rs2,550mn in Q2FY20 due to lower effective tax rate, better operational performance, higher other income and exceptional gains. Despite challenges in the demand environment, 2HFY20 is expected to be better than 1HFY20 with reasonable growth in exports and presence of levers for margin improvement. Despite lower volumes in Q2FY20, commodity prices softened and margins improved. Sentiments are improving and outlook is positive for Q4FY20 and Q1FY21. Exports are contributing well and TVS is all set for a smooth BS6 transition. We assign an Accumulate rating with a target price of Rs. 474.

 This article provides an update on the prices of various chemicals in India, including Aniline, Benzene, Toluene, Styrene Monomer, Phthalic Anhydride, and Orthoxylene. The prices are provided for each mont

In [51]:
len(results_curiosity['input_documents'])

14

chainType=stuff creates a Because we used map_reduce and the document was too long for the 'stuff' chain_type, the pdf document has been split into 14 different pieces with each summarized and then a summary of summaries prepared

In [52]:
print('Summary:','\n',results_curiosity['output_text'])

Summary: 
  SMIFS Limited is a research analyst firm that provides chemical updates and specific disclosures. They do not have any financial interest in the subject company, nor do they have any equity holdings, actual/beneficial ownership, or material conflict of interest. They provide updates on the prices of various chemicals in India, including Aniline, Benzene, Toluene, Styrene Monomer, Phthalic Anhydride, and Orthoxylene. They also provide updates on the prices of Indian Maleic Anhydride, Mono Ethylene Glycol, Iso Propyl Alcohol (IPA), Phenol, Acetone, and Acetonitrile. Additionally, they provide an overview of the key raw materials used by major Indian chemical companies and a list of companies with key chemistries or competencies. They also provide an update on the global carbon black market and the domestic carbon black industry. Finally, they provide a disclaimer outlining the conditions and certifications of SMIFS Limited.


In [46]:
results_curiosity.keys()

dict_keys(['input_documents', 'output_text'])

In [60]:
for i in range(len(results_curiosity['input_documents'])):
    doc=results_curiosity['input_documents'][i].page_content
    print(f'Section {i}: Length: {len(doc)}')

Section 0: Length: 8560
Section 1: Length: 1716
Section 2: Length: 1692
Section 3: Length: 1189
Section 4: Length: 2273
Section 5: Length: 1117
Section 6: Length: 3265
Section 7: Length: 3892
Section 8: Length: 2286
Section 9: Length: 5622
Section 10: Length: 4465
Section 11: Length: 4041
Section 12: Length: 3386
Section 13: Length: 2192


##### Load the default chain_type i.e. stuff

In [63]:
summarize_chain=load_summarize_chain(llm,verbose=True)

In [64]:
summarize_chain(retriever)



> Entering new  chain...


> Entering new  chain...
Prompt after formatting:
Write a concise summary of the following:


" 
 
 
 
  
 
esult Update  SMIFS Limited  1 
esult Update  SMIFS Limited  1  
 
     
            
                  INSTITUTIONAL RESEARCH  
 
Chemical Update
  
 1 SMIFS Research is also available on Bloomberg
    
|Price performance (%)*  
 1M 3M 12M  36M  
S&P BSE 500  -3.1 4.3 3.5 55.6  
Aarti Industries  -9.5 -18.1  -39.3  45.2  
SRF  -4.3 -8.3 -5.2 231.4  
Navin Fluorine  -7.5 -9.2 -2.5 306.2 
Rossari Biotech  -10.4  -24.6  -43.0  0.0 
NOCIL  0.3 -7.4 0.8 113.5  
IGPL  -3.3 -8.7 -25.8  198.4  
Oriental Carbon  1.5 -7.6 -18.6  -22.6  
Bodal Chemical  -2.1 -8.5 -29.3  7.3 
Valiant Organic  -10.8  -19.7  -43.2  -9.0 
Deepak Fert . -11.9  -19.7  89.5  645.3  
Deepak Nitrite  -10.2  -2.4 -20.2  416.6  
Thirumalai Che m. -1.0 -5.2 -14.3  180.3  
GNFC  -6.6 -13.4  26.2  230.8  
India Glycol  -1.2 -7.9 -15.1  231.1  
DCM Shriram  0.5 -10.8  -7.1 124.6  
*as on 02nd

InvalidRequestError: This model's maximum context length is 4097 tokens, however you requested 13249 tokens (12993 in your prompt; 256 for the completion). Please reduce your prompt; or completion length.

By default, the chain_type is set as stuff. However, this chain_type tries to process the entire document at one time which leads to an error:<br><br>
    <b>InvalidRequestError:</b> This model's maximum context length is 4097 tokens, however you requested 13249 tokens (12993 in your prompt; 256 for the completion). Please reduce your prompt; or completion length.

## Question Answering using Prompts 

#### Create prompt

In [67]:
from langchain.prompts import PromptTemplate

In [66]:
template='Answer the questions posed as Question: {question}\n\nAnswer:'
inputs=['question']

In [68]:
prompt=PromptTemplate(
    template=template,\
    input_variables=inputs
)

#### Create the LLM

In [69]:
from langchain.llms import OpenAI

In [70]:
llm=OpenAI(temperature=0,model='text-davinci-003')

#### Run the user query

In [71]:
user_query1='Why is the sky colored blue?'

In [80]:
qa_result=llm(prompt.format_prompt(question=user_query1).to_string())

In [82]:
print(qa_result)

 The sky is colored blue because of the way sunlight interacts with the atmosphere. The blue color is caused by the scattering of sunlight off of molecules in the atmosphere, which is known as Rayleigh scattering.


In [83]:
user_query2='What is the meaning of life?'

In [84]:
qa_result2=llm(prompt.format_prompt(question=user_query2).to_string())

In [85]:
print(qa_result2)

 The meaning of life is subjective and can vary from person to person. Ultimately, it is up to each individual to decide what the meaning of life is for them.
